### The **Tricks** I should have known earlier :(

### The **Question** I have asked 1k+ times on StackOverflow ;)

- Saving images in Python at a very high quality in `matplotlib`, [here](https://stackoverflow.com/questions/16183462/saving-images-in-python-at-a-very-high-quality).

- Fast method to retrieve contour mask from a binary mask in Python using `cv2`, [here](https://stackoverflow.com/questions/40441910/fast-method-to-retrieve-contour-mask-from-a-binary-mask-in-python).